## Explore, segment, and cluster the neighborhoods in the city of Toronto

In [50]:
import requests
import pandas as pd
import numpy as np

# import k-means from clustering stage
from sklearn.cluster import KMeans

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

wikipedia_url = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
#wikipedia_url

# <b><u>Part 1</u></b>
### Page scraping and parse the table
I planned to use BeautifulSoup4 so we need to install the required module.lxml, et_xmlfile, and BeautifulSoup4


In [4]:
!conda install -c conda-forge lxml --yes
!conda install -c conda-forge et_xmlfile --yes

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.10

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - lxml


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    lxml-4.4.0                 |   py36h7ec2d77_0         1.6 MB  conda-forge

The following packages will be UPDATED:

    lxml: 4.2.5-py37hefd8a0e_0 --> 4.4.0-py36h7ec2d77_0 conda-forge


lxml-4.4.0           | 1.6 MB    | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done
Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.10

Please update conda by running

    $ cond

In [5]:
!conda install -c conda-forge BeautifulSoup4 --yes                         
#if update is required for the module 
#!conda update -n base -c defaults BeautifulSoup4 --yes


Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.10

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - beautifulsoup4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    soupsieve-1.9.2            |           py36_0          59 KB  conda-forge
    beautifulsoup4-4.8.0       |           py36_0         144 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         202 KB

The following NEW packages will be INSTALLED:

    soupsieve:      1.9.2-py36_0 conda-forge

The following packages will be UPDATED:

    beautifulsoup4: 4.6.3-py37_0             --> 4.8.0-py36_0 conda-forge


soupsieve-1.9.2     

In [51]:
#read the page and print the soup 
from bs4 import BeautifulSoup
soup = BeautifulSoup(wikipedia_url,"lxml")
#print(soup.prettify())

## Retrive the required information from page 
I have create soup object now we can traverse and extract the required information from the content

In [7]:
My_table = soup.find('table',{'class':'wikitable sortable'})

#creating the list of headers which can be used for column names of dataframe
headers = soup.table.findAll('th')
header_list = []
for k, header in enumerate(headers):
    (header_list.append(header.text.strip()))
header_list

['Postcode', 'Borough', 'Neighbourhood']

In [8]:
row_list = []
for i, row in enumerate(soup.table.findAll("tr")):
    cols_values = []
    for j,col in enumerate(row.findAll('td')):
        cols_values.append(col.text.strip())
        row_list.append(cols_values)
df1 = pd.DataFrame(row_list)
df1.columns= header_list

#remove all Not assigned Borough form the dataframe
df1 = df1[(df1["Borough"]!= 'Not assigned')]
#df1.loc[df1["Neighbourhood"]== 'Not assigned',["Neighbourhood"]] 
#df1.loc[df1["Neighbourhood"]== 'Not assigned',['Borough']]


In [9]:
for i, x in df1.loc[df1["Neighbourhood"]== 'Not assigned',["Neighbourhood"]].iterrows() :
    #print(i)
    df1.loc[i,"Neighbourhood"] = df1.loc[i,"Borough"]


In [10]:
df1.loc[df1["Neighbourhood"]== 'Not assigned',["Neighbourhood"]]

,Neighbourhood


## Merge all the Neighbourhood values to one cell 
After groupby postcode we need to aggregate unique values of neighbourhood 

In [13]:
df2 = df1.groupby(by=['Postcode','Borough']).agg(lambda x: ','.join(x.unique()))
df2.reset_index(level=['Postcode','Borough'], inplace=True)
df2.head(103)

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park"
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge"
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff,Cliffside West"


In [14]:
df2.shape

(103, 3)

# <b><u>Part 2</u></b> 
### Adding the location data to the dataframe 

In [14]:
!conda install -c conda-forge geocoder --yes 
import geocoder

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.10

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - geocoder


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    click-7.0                  |             py_0          61 KB  conda-forge
    ratelim-0.1.6              |             py_2           6 KB  conda-forge
    geocoder-1.38.1            |             py_1          53 KB  conda-forge
    future-0.17.1              |        py36_1000         701 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         821 KB

The following NEW packages will be INSTALLED:

    future:   0.17.1-py36_1000 co

In [15]:
# this api never works i tried just to test but while loop did not yeild 
# any results after even after running the while loop for more than 5 mins
#geo_inf = None
#while (geo_inf is None):
 #   g = geocoder.google('Mountain View, CA')
 #   geo_inf = g.latlng

In [16]:
geospatial_data = pd.read_csv("https://cocl.us/Geospatial_data")


In [17]:
geo_t_df = pd.concat([df2,geospatial_data],axis=1,sort=False)


In [18]:
#removing the duplicate columns from concatinated dataframe
geo_t_df = geo_t_df.T.drop_duplicates().T
geo_t_df

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.8067,-79.1944
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.7845,-79.1605
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.7636,-79.1887
3,M1G,Scarborough,Woburn,43.771,-79.2169
4,M1H,Scarborough,Cedarbrae,43.7731,-79.2395
5,M1J,Scarborough,Scarborough Village,43.7447,-79.2395
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.7279,-79.262
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.7111,-79.2846
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.7163,-79.2395
9,M1N,Scarborough,"Birch Cliff,Cliffside West",43.6927,-79.2648


In [21]:
explore_df = geo_t_df[geo_t_df["Borough"].str.contains(pat = "Toronto")]

In [23]:

explore_df

,Postcode,Borough,Neighbourhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.6764,-79.293
41,M4K,East Toronto,"The Danforth West,Riverdale",43.6796,-79.3522
42,M4L,East Toronto,"The Beaches West,India Bazaar",43.669,-79.3156
43,M4M,East Toronto,Studio District,43.6595,-79.3409
44,M4N,Central Toronto,Lawrence Park,43.728,-79.3888
45,M4P,Central Toronto,Davisville North,43.7128,-79.3902
46,M4R,Central Toronto,North Toronto West,43.7154,-79.4057
47,M4S,Central Toronto,Davisville,43.7043,-79.3888
48,M4T,Central Toronto,"Moore Park,Summerhill East",43.6896,-79.3832
49,M4V,Central Toronto,"Deer Park,Forest Hill SE,Rathnelly,South Hill,...",43.6864,-79.4


# <b><u> Part 3</u></b>
### Explore and cluster the neighborhoods in Toronto

In [24]:
!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.10

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          90 KB

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0   conda-forge
    geopy:         1.20.0-py_0 conda-forge


geopy-1.20.0         | 57 KB     | ##################################### | 100% 
geographiclib-1.49   | 

In [25]:
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

In [26]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Tornoto City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Tornoto City are 43.653963, -79.387207.


In [27]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, borough, neighborhood in zip(explore_df['Latitude'], explore_df['Longitude'], explore_df['Borough'], explore_df['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [28]:
CLIENT_ID = 'MA4TCT35GORIHS4EEZUC451QKTZKQWE332JNV21ZXULHPDN2' # your Foursquare ID
CLIENT_SECRET = 'IJSWBANV1BAIHU4O3F5DNNRX1SJOIXKY3KN2UCMYW3A1IQV2' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
radius = 500
LIMIT = 100
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: MA4TCT35GORIHS4EEZUC451QKTZKQWE332JNV21ZXULHPDN2
CLIENT_SECRET:IJSWBANV1BAIHU4O3F5DNNRX1SJOIXKY3KN2UCMYW3A1IQV2


In [29]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [30]:
toronto_nearby_venues = getNearbyVenues(names=explore_df['Neighbourhood'],
                                   latitudes=explore_df['Latitude'],
                                   longitudes=explore_df['Longitude']
                                  )


The Beaches
The Danforth West,Riverdale
The Beaches West,India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park,Summerhill East
Deer Park,Forest Hill SE,Rathnelly,South Hill,Summerhill West
Rosedale
Cabbagetown,St. James Town
Church and Wellesley
Harbourfront,Regent Park
Ryerson,Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide,King,Richmond
Harbourfront East,Toronto Islands,Union Station
Design Exchange,Toronto Dominion Centre
Commerce Court,Victoria Hotel
Roselawn
Forest Hill North,Forest Hill West
The Annex,North Midtown,Yorkville
Harbord,University of Toronto
Chinatown,Grange Park,Kensington Market
CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place,Underground city
Christie
Dovercourt Village,Dufferin
Little Portugal,Trinity
Brockton,Exhibition Place,Parkdale Village
High Park,The Junction South
Parkdale,Roncesvall

In [31]:
toronto_nearby_venues

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,"The Danforth West,Riverdale",43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant
5,"The Danforth West,Riverdale",43.679557,-79.352188,Dolce Gelato,43.677773,-79.351187,Ice Cream Shop
6,"The Danforth West,Riverdale",43.679557,-79.352188,MenEssentials,43.677820,-79.351265,Cosmetics Shop
7,"The Danforth West,Riverdale",43.679557,-79.352188,Cafe Fiorentina,43.677743,-79.350115,Italian Restaurant
8,"The Danforth West,Riverdale",43.679557,-79.352188,La Diperie,43.677530,-79.352295,Ice Cream Shop
9,"The Danforth West,Riverdale",43.679557,-79.352188,Messini Authentic Gyros,43.677827,-79.350569,Greek Restaurant


In [32]:
toronto_nearby_venues.groupby('Neighborhood').count()
toronto_nearby_venues.shape

(1689, 7)

In [33]:
print('There are {} uniques categories.'.format(len(toronto_nearby_venues['Venue Category'].unique())))

There are 235 uniques categories.


In [34]:
# one hot encoding
toronto_nearby_venues_onehot = pd.get_dummies(toronto_nearby_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_nearby_venues_onehot["Neighbourhood"] = toronto_nearby_venues["Neighborhood"] 

# move neighborhood column to the first column
fixed_columns = [toronto_nearby_venues_onehot.columns[-1]] + list(toronto_nearby_venues_onehot.columns[:-1])
toronto_nearby_venues_onehot = toronto_nearby_venues_onehot[fixed_columns]

 
toronto_nearby_venues_onehot.head()


,Neighbourhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Yoga Studio
0,The Beaches,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"The Danforth West,Riverdale",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [35]:
toronto_nearby_venues_onehot.shape


(1689, 236)

In [36]:
toronto_grouped = toronto_nearby_venues_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped

,Neighbourhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Yoga Studio
0,"Adelaide,King,Richmond",0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.030000,0.000000,...,0.00000,0.00000,0.00,0.010000,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000
1,Berczy Park,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,...,0.00000,0.00000,0.00,0.017857,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,"Brockton,Exhibition Place,Parkdale Village",0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,...,0.00000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,Business Reply Mail Processing Centre 969 Eastern,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,...,0.00000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.055556
4,"CN Tower,Bathurst Quay,Island airport,Harbourf...",0.000000,0.0625,0.0625,0.0625,0.125,0.1875,0.125,0.000000,0.000000,...,0.00000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,"Cabbagetown,St. James Town",0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,...,0.00000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Central Bay Street,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.012195,0.000000,...,0.00000,0.00000,0.00,0.012195,0.000000,0.000000,0.012195,0.000000,0.000000,0.012195
7,"Chinatown,Grange Park,Kensington Market",0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,...,0.00000,0.00000,0.00,0.060000,0.000000,0.040000,0.010000,0.000000,0.000000,0.000000
8,Christie,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,...,0.00000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,Church and Wellesley,0.012048,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.012048,0.000000,...,0.00000,0.00000,0.00,0.000000,0.012048,0.012048,0.000000,0.000000,0.012048,0.012048


In [37]:
toronto_grouped.shape[0]

38

In [38]:
num_top_venues = 5

for hood in toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide,King,Richmond----
             venue  freq
0      Coffee Shop  0.07
1             Café  0.05
2       Steakhouse  0.04
3              Bar  0.04
4  Thai Restaurant  0.04


----Berczy Park----
                venue  freq
0         Coffee Shop  0.09
1        Cocktail Bar  0.05
2              Bakery  0.05
3      Farmers Market  0.04
4  Seafood Restaurant  0.04


----Brockton,Exhibition Place,Parkdale Village----
            venue  freq
0     Coffee Shop  0.10
1            Café  0.10
2  Breakfast Spot  0.10
3      Restaurant  0.05
4    Climbing Gym  0.05


----Business Reply Mail Processing Centre 969 Eastern----
         venue  freq
0  Yoga Studio  0.06
1   Restaurant  0.06
2          Spa  0.06
3      Brewery  0.06
4  Pizza Place  0.06


----CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara----
                venue  freq
0     Airport Service  0.19
1      Airport Lounge  0.12
2    Airport Terminal  0.12
3    Sculpture Garden  

In [39]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [40]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Coffee Shop,Café,Bar,Steakhouse,Thai Restaurant,Gym,Asian Restaurant,Breakfast Spot,Burger Joint,Hotel
1,Berczy Park,Coffee Shop,Bakery,Cocktail Bar,Farmers Market,Steakhouse,Seafood Restaurant,Café,Cheese Shop,Beer Bar,Park
2,"Brockton,Exhibition Place,Parkdale Village",Breakfast Spot,Café,Coffee Shop,Performing Arts Venue,Intersection,Burrito Place,Stadium,Caribbean Restaurant,Restaurant,Bar
3,Business Reply Mail Processing Centre 969 Eastern,Yoga Studio,Auto Workshop,Pizza Place,Gym / Fitness Center,Recording Studio,Restaurant,Burrito Place,Skate Park,Smoke Shop,Brewery
4,"CN Tower,Bathurst Quay,Island airport,Harbourf...",Airport Service,Airport Lounge,Airport Terminal,Coffee Shop,Airport,Airport Food Court,Airport Gate,Bar,Harbor / Marina,Sculpture Garden


In [41]:
# set number of clusters
kclusters = 4

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=23,n_init=20).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [42]:
#toronto_grouped_clustering
toronto_grouped

,Neighbourhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Yoga Studio
0,"Adelaide,King,Richmond",0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.030000,0.000000,...,0.00000,0.00000,0.00,0.010000,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000
1,Berczy Park,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,...,0.00000,0.00000,0.00,0.017857,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,"Brockton,Exhibition Place,Parkdale Village",0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,...,0.00000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,Business Reply Mail Processing Centre 969 Eastern,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,...,0.00000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.055556
4,"CN Tower,Bathurst Quay,Island airport,Harbourf...",0.000000,0.0625,0.0625,0.0625,0.125,0.1875,0.125,0.000000,0.000000,...,0.00000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,"Cabbagetown,St. James Town",0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,...,0.00000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Central Bay Street,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.012195,0.000000,...,0.00000,0.00000,0.00,0.012195,0.000000,0.000000,0.012195,0.000000,0.000000,0.012195
7,"Chinatown,Grange Park,Kensington Market",0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,...,0.00000,0.00000,0.00,0.060000,0.000000,0.040000,0.010000,0.000000,0.000000,0.000000
8,Christie,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,...,0.00000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,Church and Wellesley,0.012048,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.012048,0.000000,...,0.00000,0.00000,0.00,0.000000,0.012048,0.012048,0.000000,0.000000,0.012048,0.012048


In [43]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

#manhattan_merged = manhattan_data
toronto_merged = explore_df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

toronto_merged.head() # check the last columns!

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,M4E,East Toronto,The Beaches,43.6764,-79.293,2,Trail,Pub,Neighborhood,Health Food Store,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Dive Bar
41,M4K,East Toronto,"The Danforth West,Riverdale",43.6796,-79.3522,0,Greek Restaurant,Coffee Shop,Ice Cream Shop,Italian Restaurant,Furniture / Home Store,Yoga Studio,Bookstore,Brewery,Bubble Tea Shop,Caribbean Restaurant
42,M4L,East Toronto,"The Beaches West,India Bazaar",43.669,-79.3156,0,Pizza Place,Italian Restaurant,Pet Store,Gym,Coffee Shop,Pub,Movie Theater,Sandwich Place,Burrito Place,Burger Joint
43,M4M,East Toronto,Studio District,43.6595,-79.3409,0,Café,Coffee Shop,Gastropub,Italian Restaurant,American Restaurant,Bakery,Yoga Studio,Convenience Store,Seafood Restaurant,Sandwich Place
44,M4N,Central Toronto,Lawrence Park,43.728,-79.3888,3,Park,Lake,Bus Line,Swim School,Yoga Studio,Doner Restaurant,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant


In [44]:
neighborhoods_venues_sorted

,Cluster Labels,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,"Adelaide,King,Richmond",Coffee Shop,Café,Bar,Steakhouse,Thai Restaurant,Gym,Asian Restaurant,Breakfast Spot,Burger Joint,Hotel
1,0,Berczy Park,Coffee Shop,Bakery,Cocktail Bar,Farmers Market,Steakhouse,Seafood Restaurant,Café,Cheese Shop,Beer Bar,Park
2,0,"Brockton,Exhibition Place,Parkdale Village",Breakfast Spot,Café,Coffee Shop,Performing Arts Venue,Intersection,Burrito Place,Stadium,Caribbean Restaurant,Restaurant,Bar
3,0,Business Reply Mail Processing Centre 969 Eastern,Yoga Studio,Auto Workshop,Pizza Place,Gym / Fitness Center,Recording Studio,Restaurant,Burrito Place,Skate Park,Smoke Shop,Brewery
4,0,"CN Tower,Bathurst Quay,Island airport,Harbourf...",Airport Service,Airport Lounge,Airport Terminal,Coffee Shop,Airport,Airport Food Court,Airport Gate,Bar,Harbor / Marina,Sculpture Garden
5,0,"Cabbagetown,St. James Town",Coffee Shop,Café,Restaurant,Italian Restaurant,Pub,Bakery,Market,Pizza Place,Park,Pet Store
6,0,Central Bay Street,Coffee Shop,Ice Cream Shop,Italian Restaurant,Café,Burger Joint,Sandwich Place,Salad Place,Indian Restaurant,Bubble Tea Shop,Spa
7,0,"Chinatown,Grange Park,Kensington Market",Vegetarian / Vegan Restaurant,Café,Chinese Restaurant,Coffee Shop,Mexican Restaurant,Vietnamese Restaurant,Dumpling Restaurant,Bar,Bakery,Comfort Food Restaurant
8,0,Christie,Café,Grocery Store,Park,Restaurant,Convenience Store,Nightclub,Baby Store,Italian Restaurant,Coffee Shop,Athletics & Sports
9,0,Church and Wellesley,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Gay Bar,Restaurant,Men's Store,Hotel,Gastropub,Fast Food Restaurant,Mediterranean Restaurant


In [45]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [46]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
41,East Toronto,0,Greek Restaurant,Coffee Shop,Ice Cream Shop,Italian Restaurant,Furniture / Home Store,Yoga Studio,Bookstore,Brewery,Bubble Tea Shop,Caribbean Restaurant
42,East Toronto,0,Pizza Place,Italian Restaurant,Pet Store,Gym,Coffee Shop,Pub,Movie Theater,Sandwich Place,Burrito Place,Burger Joint
43,East Toronto,0,Café,Coffee Shop,Gastropub,Italian Restaurant,American Restaurant,Bakery,Yoga Studio,Convenience Store,Seafood Restaurant,Sandwich Place
46,Central Toronto,0,Sporting Goods Shop,Coffee Shop,Clothing Store,Yoga Studio,Mexican Restaurant,Diner,Dessert Shop,Park,Chinese Restaurant,Restaurant
47,Central Toronto,0,Dessert Shop,Sandwich Place,Pizza Place,Pharmacy,Coffee Shop,Restaurant,Café,Sushi Restaurant,Italian Restaurant,Indian Restaurant
49,Central Toronto,0,Coffee Shop,Pub,Pizza Place,Bagel Shop,Restaurant,Liquor Store,Sports Bar,Supermarket,Sushi Restaurant,Fried Chicken Joint
51,Downtown Toronto,0,Coffee Shop,Café,Restaurant,Italian Restaurant,Pub,Bakery,Market,Pizza Place,Park,Pet Store
52,Downtown Toronto,0,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Gay Bar,Restaurant,Men's Store,Hotel,Gastropub,Fast Food Restaurant,Mediterranean Restaurant
53,Downtown Toronto,0,Coffee Shop,Café,Park,Pub,Bakery,Restaurant,Breakfast Spot,Mexican Restaurant,Ice Cream Shop,Chocolate Shop
54,Downtown Toronto,0,Coffee Shop,Clothing Store,Cosmetics Shop,Fast Food Restaurant,Middle Eastern Restaurant,Café,Japanese Restaurant,Tea Room,Restaurant,Ramen Restaurant


In [47]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
63,Central Toronto,1,Ice Cream Shop,Garden,Yoga Studio,Dog Run,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant


In [48]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,East Toronto,2,Trail,Pub,Neighborhood,Health Food Store,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Dive Bar


In [49]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
44,Central Toronto,3,Park,Lake,Bus Line,Swim School,Yoga Studio,Doner Restaurant,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant
45,Central Toronto,3,Breakfast Spot,Park,Clothing Store,Hotel,Food & Drink Shop,Gym,Sandwich Place,Falafel Restaurant,Farmers Market,Event Space
48,Central Toronto,3,Park,Restaurant,Playground,Yoga Studio,Dive Bar,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant
50,Downtown Toronto,3,Park,Playground,Building,Trail,Dive Bar,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant
64,Central Toronto,3,Park,Trail,Sushi Restaurant,Jewelry Store,Yoga Studio,Dog Run,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space


### The biggest cluster has most number of Coffee Shops and Cafe' in common